In [1]:
from __future__ import annotations

import time
import datetime
import os
import json

import cst_python as cst
import toolpy as tp
from toolpy.integrations import groq

import gbyg as gg

In [2]:
groq_interface = groq.GroqInterface(model=groq.GroqModel.LLAMA3_70B, n_retry=5)

registry = tp.llm.LLMRegistry()
registry.registry(model_name="llama3-70b", interface=groq_interface, default=True)

In [3]:
importance_threshould = 1

In [4]:
mind = cst.Mind()

gg.agent.agent_constructor(mind, importance_threshould)

for codelet in mind.code_rack.all_codelets:
    codelet.time_step = 100

memories_input = mind.raw_memory.get_all_of_type("MemoriesInput")[0]
memory_stream = mind.raw_memory.get_all_of_type("MemoryStream")[0]
agent_time_memory = mind.raw_memory.get_all_of_type("AgentTime")[0]
agent_info_memory = mind.raw_memory.get_all_of_type("AgentInfo")[0]
agent_summary_description = mind.raw_memory.get_all_of_type("AgentSummaryDescription")[0]
actual_place_memory = mind.raw_memory.get_all_of_type("ActualPlace")[0]
known_world = mind.raw_memory.get_all_of_type("KnownWorld")[0]
action_memory = mind.raw_memory.get_all_of_type("Action")[0]
current_observation_memory = mind.raw_memory.get_all_of_type("CurrentObservation")[0]
current_state_memory = mind.raw_memory.get_all_of_type("CurrentState")[0]

previous_day_summary = mind.raw_memory.get_all_of_type("PreviousDaySummary")[0]
plan = mind.raw_memory.get_all_of_type("Plan")[0]

In [5]:
agent_info_memory.set_info({"name":"Alex", "age":25, "traits":"friendly, outgoing, hospitable"})
agent_time_memory.set_info(datetime.datetime.fromisoformat("2024-06-26T00:00:00").timestamp())
actual_place_memory.set_info("Bedroom")

-1

In [6]:
mind.start()

In [7]:
input_memories  = [
  {
    "index": 0,
    "type": "event",
    "created": "2024-06-01 08:00:00",
    "description": "Alex attended a nature photography workshop in the city park.",
  },
  {
    "index": 1,
    "type": "reflection",
    "created": "2024-06-01 19:00:00",
    "description": "Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects.",
  },
  {
    "index": 2,
    "type": "event",
    "created": "2024-06-05 10:00:00",
    "description": "Alex completed a major software development project at work, delivering the final product to the client.",
  },
  {
    "index": 3,
    "type": "reflection",
    "created": "2024-06-05 20:00:00",
    "description": "After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome.",
  },
  {
    "index": 4,
    "type": "event",
    "created": "2024-06-10 16:00:00",
    "description": "Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes.",
  }
]

for m in input_memories:
    m["created"] = datetime.datetime.fromisoformat(m["created"]).timestamp()
    m["last_acessed"] = m["created"]

In [8]:
memories_input_queue = memories_input.get_info()
for memory in input_memories:
    memories_input_queue.append(memory)

while len(memory_stream.get_info()) != len(input_memories):
    time.sleep(0.1)

D:\Github\GDataByGAgents\src\gbyg\agent\retrieval\recency_scorer_tool.py:32: RuntimeWarning: invalid value encountered in divide
  scores = (scores-min_score)/(max_score-min_score)
c:\Python38\lib\site-packages\sentence_transformers\util.py:41: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:278.)
  b = torch.tensor(b)


In [9]:
for memory in memory_stream.get_info():
    because_of = ""
    if 'because_of' in memory: 
        because_of = memory['because_of']

    memory_str = f"{memory['index']} ({memory['type']}): {memory['description']} | because of: {because_of}"
    print(memory_str)

0 (event): Alex attended a nature photography workshop in the city park. | because of: 
1 (reflection): Reflecting on the new techniques learned during the workshop, considering how to apply them in future projects. | because of: 
2 (event): Alex completed a major software development project at work, delivering the final product to the client. | because of: 
3 (reflection): After delivering the project, Alex felt a sense of relief and accomplishment, thinking about the challenges overcome. | because of: 
4 (event): Alex went on a weekend trip to explore a national park, taking photos of the wildlife and landscapes. | because of: 


In [10]:
last_summary_time = agent_summary_description.get_timestamp()

agent_time_memory.set_info(agent_time_memory.get_info()+gg.time_utils.TimeInterval.ONE_DAY)

previous_day_summary_content = {"summary":'''Yesteday, 26 June, Alex went to the park, where he participated in a photography workshop. 
Alex took several photos, preparing to later edit them and post them on social media.
Alex now plans to stay home the entire day.''',
                        "generated_time":agent_time_memory.get_info()}

previous_day_summary.set_info(previous_day_summary_content)

while agent_summary_description.get_timestamp() == last_summary_time:
    time.sleep(0.1)

print(agent_summary_description.get_info()["summary"])

Name: Alex (age: 25)
Innate traits: friendly, outgoing, hospitable
Alex is an outdoorsy and creative person with an interest in nature and photography.
Alex is likely a photographer, possibly a nature photographer.
Alex seems enthusiastic and possibly revitalized.


In [11]:
while plan.get_info() == "":
    time.sleep(0.1) 

In [12]:
#Remove fisrt hour actions to avoid infinity loop bellow
while plan.get_info()[0]["hour"] == 0:
    del plan.get_info()[0]

In [13]:
for action in plan.get_info():
    hour = action["hour"]
    minute = action["minute"]
    action_action = action["action"]

    print(f"{hour}:{minute}: {action_action}")

8:0: Wakes up
8:10: Has breakfast
9:0: Edits photos taken at the park
12:0: Start lunch break
12:30: Continue lunch break
13:0: Posts edited photos on social media for 60 minutes
14:0: Takes a short nap
15:0: Gathers photography equipment
15:10: Inspects photography equipment
15:20: Packs photography equipment
15:30: Double-checks photography equipment
15:40: Finishes organizing photography equipment
18:0: Getting ready for dinner
18:10: Eating dinner
18:30: Finishing up dinner and cleaning up
19:0: Watches a nature documentary
19:10: Watches a nature documentary
19:20: Watches a nature documentary
19:30: Watches a nature documentary
19:40: Watches a nature documentary
19:50: Watches a nature documentary
22:0: Retires to bed


In [14]:
alex_house = {
    "Alex's House": {
        "Living Room": [
            {"name": "sofa", 'type':'inanimate_object', 
             "description": "A comfortable, modern sofa perfect for relaxing or hosting friends.",
             "state":"idle and clean"},
            {"name": "coffee table", 'type':'inanimate_object', 
             "description": "A sleek coffee table with a few programming books and the latest tech magazines.",
             "state":"idle and clean"},
            {"name": "television", 'type':'inanimate_object', 
             "description": "A large smart TV, often used to watch tech talks or unwind with a show.",
             "state":"off and clean"},
            {"name": "bookshelf", 'type':'inanimate_object', 
             "description": "A bookshelf filled with a mix of classic literature and technical manuals.",
             "state":"idle and clean"},
            {"name": "rug", 'type':'inanimate_object', 
             "description": "A soft rug that adds a touch of warmth to the living room.",
             "state":"idle and clean"}
        ],
        "Kitchen": [
            {"name": "refrigerator", 'type':'inanimate_object', 
             "description": "A high-tech refrigerator that keeps Alex's meals and snacks fresh.",
             "state":"on and clean"},
            {"name": "stove", 'type':'inanimate_object', 
             "description": "A modern stove where Alex occasionally experiments with new recipes.",
             "state":"off and clean"},
            {"name": "microwave", 'type':'inanimate_object', 
             "description": "A quick and efficient microwave for heating up meals.",
             "state":"off and clean"},
            {"name": "kitchen sink", 'type':'inanimate_object', 
             "description": "A stainless steel sink",
             "state":"idle and clean"},
            {"name": "dishwasher", 'type':'inanimate_object', 
             "description": "A quiet dishwasher that handles the post-dinner cleanup.",
             "state":"off and clean"},
            {"name": "dining table", 'type':'inanimate_object', 
             "description": "A minimalist dining table where Alex enjoys breakfast and dinner.",
             "state":"idle and clean"},
            {"name": "chairs", 'type':'inanimate_object', 
             "description": "Comfortable chairs around the dining table, perfect for meals or working on a laptop.",
             "state":"idle and clean"}
        ],
        "Bedroom": [
            {"name": "bed", 'type':'inanimate_object', 
             "description": "A cozy bed where Alex gets his much-needed rest.",
             "state":"idle and clean"},
            {"name": "dresser", 'type':'inanimate_object', 
             "description": "A dresser with neatly organized clothes, reflecting Alex's organized nature.",
             "state":"idle and clean"},
            {"name": "nightstand", 'type':'inanimate_object', 
             "description": "A nightstand with a lamp and a few personal items.",
             "state":"idle and clean"},
            {"name": "closet", 'type':'inanimate_object', 
             "description": "A spacious closet with casual and work clothes.",
             "state":"idle and clean"},
            {"name": "desk", 'type':'inanimate_object', 
             "description": "A small desk with a notebook and pens, used for jotting down late-night ideas.",
             "state":"idle and clean"},
            {"name": "lamp", 'type':'inanimate_object', 
             "description": "A bedside lamp for reading before bed.",
             "state":"off and clean"}
        ],
        "Bathroom": [
            {"name": "shower", 'type':'inanimate_object', 
             "description": "A modern shower with various settings for a refreshing start to the day.",
             "state":"off and clean"},
            {"name": "bathroom sink", 'type':'inanimate_object', 
             "description": "A clean sink with essential toiletries.",
             "state":"idle and clean"},
            {"name": "toilet", 'type':'inanimate_object', 
             "description": "A simple, modern toilet.",
             "state":"idle and clean"},
            {"name": "mirror", 'type':'inanimate_object', 
             "description": "A mirror above the sink, used for daily grooming.",
             "state":"idle and clean"},
            {"name": "cabinet", 'type':'inanimate_object', 
             "description": "A cabinet filled with towels and personal care items.",
             "state":"idle and clean"}
        ],
        "Home Office": [
            {"name": "desk", 'type':'inanimate_object', 
             "description": "A large desk with multiple monitors and a laptop, where Alex spends most of his working hours.",
             "state":"idle and clean"},
            {"name": "computer", 'type':'inanimate_object', 
             "description": "A powerful computer equipped with the latest software development tools.",
             "state":"off and clean"},
            {"name": "office chair", 'type':'inanimate_object', 
             "description": "An ergonomic office chair to support long coding sessions.",
             "state":"idle and clean"},
            {"name": "bookshelf", 'type':'inanimate_object', 
             "description": "A bookshelf filled with programming books and reference materials.",
             "state":"idle and clean"},
            {"name": "printer", 'type':'inanimate_object',
              "description": "A reliable printer for printing documents and designs.",
              "state":"off and clean"}
        ]
    }
}

alex = {"name":"Alex", "type":"agent", "description":"human", 'state':'sleeping in the bed.'}

alex_house["Alex's House"]["Bedroom"].append(alex)

agent_names = ["Alex"]

In [15]:
from collections import deque
from copy import deepcopy

known_world_tree = deepcopy(alex_house)

stack = deque()
for room_name in known_world_tree:
    room = known_world_tree[room_name]
    stack.append(room)

while len(stack) > 0:
    room = stack.pop()
    if isinstance(room, list):
        for object in room:
            del object["state"]
    else:
        for subroom_name in room:
            subroom = room[subroom_name]
            stack.append(subroom)

In [16]:
known_world.set_info(known_world_tree)

-1

In [17]:
observation_memory = {"index": 0,
    "type": "event",
    "created": agent_time_memory.get_info(),
    "description": f"Alex is {alex['state']}"}

input_memories.append(observation_memory)

In [18]:
action_memory.get_info()

''

In [31]:
first_action = plan.get_info()[0]
last_action = plan.get_info()[-1]

agent_time = agent_time_memory.get_info()

now = datetime.datetime.fromtimestamp(agent_time)

now = now.replace(hour=first_action["hour"], minute=first_action["minute"])
agent_time_memory.set_info(now.timestamp())

simulation_end_time = datetime.datetime.fromtimestamp(agent_time)
simulation_end_time = simulation_end_time.replace(hour=last_action["hour"], minute=last_action["minute"]).timestamp()

In [20]:
while action_memory.get_info() == "":
    time.sleep(0.1)

last_action_time = action_memory.get_timestamp()

In [21]:
env = gg.environment.Environment(alex_house, agent_names)

In [22]:
import numpy as np

def encoder(obj):
    if isinstance(obj, np.integer):
            return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    
    return obj.__dict__

sensor_result = {}

def dump_log(path:str):
    log = {}
    log["agent_time"] = agent_time_memory.get_info()
    log["action"] = action_memory.get_info()
    log["agent_summary_description"] = agent_summary_description.get_info()
    log["state"] = current_state_memory.get_info()
    log["curret_observation"] = current_observation_memory.get_info()
    log["current_place"] = actual_place_memory.get_info()
    log["sensor_result"] = sensor_result
    log["environment_tree"] = env._environment_tree
    

    log["memory_stream"] = list(memory_stream.get_info())

    if not os.path.isdir(path):
        os.mkdir(path)

    log_path = os.path.join(path, str(log["agent_time"])+".json")
    file = open(log_path, "w")
    json.dump(log, file, default=encoder)
    file.close()


In [23]:
dump_log("logs")

In [24]:
sound_sensor = gg.sensors.SoundSensor()

In [25]:
while agent_time_memory.get_info() < simulation_end_time:

    agent_actions = {"Alex":action_memory.get_info()}

    obsevation = env.step(agent_actions)
    alex_observation = obsevation["Alex"]

    last_memory_stream_size = len(memory_stream.get_info())

    observation_memory = {"type": "event",
        "created": agent_time_memory.get_info(),
        "description": f"Alex is {alex_observation['state']}"}

    memories_input_queue.append(observation_memory)

    observation_memory = {"type": "event",
        "created": agent_time_memory.get_info(),
        "description": alex_observation['observation']}

    memories_input_queue.append(observation_memory)

    current_state_memory.set_info(alex_observation["state"])
    current_observation_memory.set_info(alex_observation['observation'])
    actual_place_memory.set_info(alex_observation["place"])

    current_time = agent_time_memory.get_info()
    current_time += alex_observation["duration"]
    agent_time_memory.set_info(current_time)

    #Get sensor output
    query = {"action":agent_actions["Alex"],
            "state":alex_observation['state'],
            "observation":alex_observation['observation']}

    sensor_result, _ = sound_sensor(query)

    #Wait for agent processing
    while action_memory.get_timestamp() == last_action_time or len(memory_stream.get_info()) < last_memory_stream_size+2:
        time.sleep(0.1)

    last_action_time = action_memory.get_timestamp()

    dump_log("logs")